In [1]:
include("src/lisa_store.jl")

using ..Store
using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using HDF5
using EasyConfig

db = Graph.DB("lisa.db")
hll = SetCore.HllSet{10}()

# Create an empty HDF5 file. Run it only once, because it will overwrite the file
h5open("lisa_arch.hdf5", "cw") do f
    # The file is now open, but it's empty
end

In [2]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/")

In [3]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row)
    col_uuid = string(uuid4())
    Store.ingest_csv_by_column(db, assign, col_uuid; limit=100, offset=10)
end

Processed column: 8
Processed column: 15


In [4]:
Store.commit(db, "lisa_arch.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", "commit 2", Config())

row.props: {"file_name":"/home/alexmy/JULIA/DEMO/sample/Isc_london-ksi-only-since2010.csv","file_type":"csv","commit_id":"1c2a66db-a673-40de-b2ce-b9fb548b2eed"}
node.props: {"file_name":"/home/alexmy/JULIA/DEMO/sample/Isc_london-ksi-only-since2010.csv","file_type":"csv","commit_id":"2da19026-e0ca-4729-86af-d183f5aec3ea"}
Exporting node: 0b90b1fee69c77ffa3efe57db7788112ef96dba6
file: true
row.props: {"column_name":"Severity of casualty","file_sha1":"0b90b1fee69c77ffa3efe57db7788112ef96dba6","column_type":"String","commit_id":"1c2a66db-a673-40de-b2ce-b9fb548b2eed"}
node.props: {"column_name":"Severity of casualty","file_sha1":"0b90b1fee69c77ffa3efe57db7788112ef96dba6","column_type":"String","commit_id":"2da19026-e0ca-4729-86af-d183f5aec3ea"}
Exporting node: 4b925186fe6753be2ed6908976e44e0f7630d40f
file: false
row.props: {"column_name":"Casualty Class","file_sha1":"0b90b1fee69c77ffa3efe57db7788112ef96dba6","column_type":"String","commit_id":"1c2a66db-a673-40de-b2ce-b9fb548b2eed"}
node.pro

In [5]:
# arch = h5open("lisa_arch.hdf5")

# println(names(arch))

function print_hdf5_tree(obj, indent="", limit = 100)
    i = 0
    for name in names(obj)
        if i > limit
            println(indent, "...")
            break
        end
        child = obj[name]
        println(indent, name)
        if isa(child, HDF5Group)
            print_hdf5_tree(child, indent * "    ")
        end
        i += 1
    end
end

h5open("lisa_arch.hdf5", "r") do file
    print_hdf5_tree(file)
end

2da19026-e0ca-4729-86af-d183f5aec3ea
    edges
        0b90b1fee69c77ffa3efe57db7788112ef96dba6
            has_column
                1d29c5326d2292d8717e189a12ca4bd4cbac8b76
                3f9526f8d331b9519b8632a11b2d344ab7c647b6
                4b925186fe6753be2ed6908976e44e0f7630d40f
                4ea0d4a979741fbb57dd4cc9a2c05913a6ee875d
                5c6ad737758c030a92351e80c15650712fa06108
                8e5610e809bad1efe796848a86ecb9e28e1709b9
                d87927e475744f6280feca8fd040dd42d07dba4a
                de6012ead358304ae31a499f29d380878219a256
        6be12bee4edf7c96016907e44bb520be80dc9232
            has_column
                0f73a283cd731d2f36ef6b029f24d28458b8fefc
                2907d275ede98697de7f5a5bfa30a174dc05314c
                34a7b3a7a81e0f1b7f357c6aad5fb1ed31df950c
                4081c0ee7a492aa0ffd6494620761a9e760fa400
                68095ecaf7005e07952f3029a21534c294b55742
                795c87e41fe35fb6530d609cb6835daa0e5fbc1b
           

In [6]:
file = h5open("lisa_arch.hdf5", "r")
println(isempty(file), " ", typeof(file))
data_out = Dict()
Store.read_datasets(file, data_out, "/23d3b589-1d7c-4e86-a0f4-8c54a322c017/nodes/")
println(data_out)
# Close the file
close(file)

false HDF5File
Dict{Any, Any}()


In [7]:
Graph.unset_lock!(db, :, :)

SQLite.Query{false}(SQLite.Stmt(SQLite.DB("lisa.db"), Base.RefValue{Ptr{SQLite.C.sqlite3_stmt}}(Ptr{SQLite.C.sqlite3_stmt} @0x000000000499ea98), Dict{Int64, Any}()), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))